In [1]:
%%time
from ipynb.fs.full.functions_sim import *
from ipynb.fs.full.function_adjust import *
from ipynb.fs.full.topologias import *
import random
#random.seed(42)
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# Initialize your graph G, channels, qubits, and other variables here.

num_qubits = 10 # Number of qubits to send
timeslot_counts = [0] * num_qubits  # Records the number of timeslots for each qubit
need_create_epr = [0] * num_qubits  # N times needed to create the EPR
num_simulations = 1

# Example usage:
rows = 3
cols = 4
num_nodes = rows * cols
#num_nodes = 1024

# Set the routing_strategy to choose the desired strategy (1, 2, or 3)
routing_strategy = 0

# Initialize a list to store average fidelity values for best routes
average_fidelities = []
average_eprs = []
average_slots = []
average_qubit_perc = []
average_eprs_perc = []
average_miss_qubit = []


#Variáveis para o Aprendizado Quântico
#num_register = 8 #Specify the number of qubits you need
#qubit_register = initialize_qubit_register(11)
#quantum_circuit = initialize_quantum_circuit(qubit_register)

alpha, gamma, epsilon = 0.1,0.95,0.3

print(f"Estratégia de Roteamento: {routing_strategy}")

for diff_nodes in range(2, 9):  # Vary diff_nodes from 2 to 8

#for diff_nodes in range(2, 3):  # Vary diff_nodes from 2 to 8
    print(f"Current diff_nodes: {diff_nodes}")
    total_fidelities = []
    total_eprs = []
    qubits_diff = []
    eprs_diff = []
    qubit_perc = []
    eprs_perc = []
    count_miss_qubit = 0
    total_slots = []

    for simulation in range(num_simulations):
        #print(f"Current sim: {simulation}")
        G, channels = create_lattice_topology_network(rows, cols)  #Usa redes diferentes em cada simulação. Restart a rede
        #G, channels = create_ring_topology_network(num_nodes)
        #G, channels = create_star_topology_network(num_nodes)
        #G, channels = create_line_topology_network(num_nodes)
        #G, channels =  create_fully_connected_topology_network(num_nodes)
        #plot_network(G, channels)
        # Rest of your simulation code remains the same
        initial_qubits, initial_eprs = estado_rede(G, channels)
        
        for qubit_index in range(num_qubits):       
            #print(f"Quibit Number: {qubit_index}")
            #plot_network(G, channels)
            #diff_nodes= random.randint(2, 6)
            sender, receiver = find_send_receiver(G,diff_nodes)
            fidelity,epr,slots = do_routing(routing_strategy, G, channels, qubit_index, sender, receiver)
                        
            if fidelity:
                total_fidelities.append(fidelity)
                total_eprs.append(epr)
                total_slots.append(slots)
                
            else:
                total_fidelities.append(0)
                total_eprs.append(0)
                total_slots.append(slots)
                count_miss_qubit  += 1
        final_qubits, final_eprs = estado_rede(G, channels)
        qubits_diff.append(initial_qubits - final_qubits)
        qubit_perc.append( (initial_qubits - final_qubits) / initial_qubits)
        eprs_diff.append(initial_eprs - final_eprs)
        eprs_perc.append( (initial_eprs - final_eprs) / initial_eprs)
    # Calculate the average initial and final differences1
    
                
    #non_zero_fidelities = [fidelity for fidelity in total_fidelities if fidelity > 0]
    #avg_fidelity = np.mean(non_zero_fidelities)
    #std_fidelity = np.std(non_zero_fidelities)
    #average_fidelities.append((avg_fidelity, std_fidelity))
    avg_fidelity = np.mean(total_fidelities)
    std_fidelity = np.std(total_fidelities)
    average_fidelities.append((avg_fidelity, std_fidelity))
    print(f"diff_node = {diff_nodes}, avg_fidelity = {avg_fidelity}, std_fidelity = {std_fidelity}")              
    
    
    
    #Agora para os EPRs
    # Filter out zeros from average_fidelities
    #non_zero_eprs = [epr for epr in total_eprs if epr > 0]
    # Calculate average and standard deviation
    #avg_eprs = np.mean(non_zero_eprs)
    #std_eprs = np.std(non_zero_eprs)
    avg_eprs = np.mean(total_eprs)
    std_eprs = np.std(total_eprs)
    #average_eprs.append((avg_eprs, std_eprs))
    average_eprs.append((avg_eprs, std_eprs))
    print(f"Average EPRs over {num_simulations} simulations: {avg_eprs}")
    print(f"Standard Deviation of EPrs over {num_simulations} simulations: {std_eprs}")
    
    avg_slots = np.mean(total_slots)
    std_slots = np.mean(total_slots)
    average_slots.append((avg_slots,std_slots))
    
    avg_qubits_perc= np.mean(qubit_perc)
    std_qubits_perc= np.std(qubit_perc)
    average_qubit_perc.append((avg_qubits_perc,std_qubits_perc))
    avg_eprs_perc= np.mean(eprs_perc)
    std_eprs_perc= np.std(eprs_perc)
    average_eprs_perc.append((avg_eprs_perc,std_eprs_perc))
    avg_count_miss_qubit = count_miss_qubit / (num_simulations * num_qubits)
    average_miss_qubit.append(avg_count_miss_qubit)
    
ger_avg_fidelity = np.mean(average_fidelities)
# After all simulations are done for different diff_nodes values:
print("\nResults for different diff_nodes:")
for idx, (avg_fidelity, std_fidelity) in enumerate(average_fidelities):
    print(f"diff_node = {idx + 2}, avg_fidelity = {avg_fidelity}, std_fidelity = {std_fidelity}")

for idx, (avg_eprs, std_eprs) in enumerate(average_eprs):
    print(f"diff_node = {idx + 2}, avg_eprs = {avg_eprs}, std_eprs = {std_eprs}")
    
    
for idx, (avg_slots, std_slots) in enumerate(average_slots):
    print(f"diff_node = {idx + 2}, avg_slots = {avg_slots}, std_slots = {std_slots}")    

print("\nConsumo da Rede for different diff_nodes:")
for idx, (avg_qubits_perc, std_qubits_perc) in enumerate(average_qubit_perc):
    print(f"diff_node = {idx + 2}, Qubit_perc = {avg_qubits_perc}, std = {std_qubits_perc}")

for idx, (avg_eprs_perc, std_eprs_perc) in enumerate(average_eprs_perc):
    print(f"diff_node = {idx + 2}, Eprs_perc = {avg_eprs_perc}, std = {std_eprs_perc}")   
    
for idx, (avg_count_miss_qubit) in enumerate(average_miss_qubit):
    print(f"diff_node = {idx + 2}, MissQubit = {avg_count_miss_qubit}, SucessRate = {(1-avg_count_miss_qubit)}")   

print(ger_avg_fidelity)

Estratégia de Roteamento: 0
Current diff_nodes: 2
diff_node = 2, avg_fidelity = 0.938450769738148, std_fidelity = 0.03839661358148382
Average EPRs over 1 simulations: 0.0
Standard Deviation of EPrs over 1 simulations: 0.0
Current diff_nodes: 3
diff_node = 3, avg_fidelity = 0.9225783497430345, std_fidelity = 0.02729518209941619
Average EPRs over 1 simulations: 0.0
Standard Deviation of EPrs over 1 simulations: 0.0
Current diff_nodes: 4
diff_node = 4, avg_fidelity = 0.9138210572833888, std_fidelity = 0.0567651447214717
Average EPRs over 1 simulations: 0.0
Standard Deviation of EPrs over 1 simulations: 0.0
Current diff_nodes: 5
diff_node = 5, avg_fidelity = 0.9210706997924175, std_fidelity = 0.03962964692636579
Average EPRs over 1 simulations: 0.0
Standard Deviation of EPrs over 1 simulations: 0.0
Current diff_nodes: 6
diff_node = 6, avg_fidelity = 0.9093545204368881, std_fidelity = 0.02914513971963566
Average EPRs over 1 simulations: 0.0
Standard Deviation of EPrs over 1 simulations: 0.0

In [2]:
import csv
# Assuming you have the following lists:
# average_fidelities, average_eprs, average_qubit_perc, average_eprs_perc, average_miss_qubit
data = []
for idx, (avg_fidelity, std_fidelity) in enumerate(average_fidelities):
    data.append({
        "diff_node": idx + 2,
        "avg_fidelity": avg_fidelity,
        "std_fidelity": std_fidelity,
        "avg_eprs": average_eprs[idx][0],
        "std_eprs": average_eprs[idx][1],
        "Qubit_perc": average_qubit_perc[idx][0],
        "std_qubits_perc": average_qubit_perc[idx][1],
        "Eprs_perc": average_eprs_perc[idx][0],
        "std_eprs_perc": average_eprs_perc[idx][1],
        "MissQubit": average_miss_qubit[idx],  # Use the appropriate value from the list
        "SucessRate": 1 - average_miss_qubit[idx],# Calculate success rate based on the list value
        "avg_slots":avg_slots,
        "std_slots":std_slots

    })
# Define the column headers
headers = ["diff_node", "avg_fidelity", "std_fidelity", "avg_eprs", "std_eprs", "Qubit_perc", "std_qubits_perc", "Eprs_perc", "std_eprs_perc", "MissQubit", "SucessRate","avg_slots","std_slots"]
file_path = "results.csv"
# Open the file for writing
with open(file_path, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headers)

    # Write the header row
    writer.writeheader()

    # Write the data rows
    writer.writerows(data)

print(f"Data saved to {file_path}")


Data saved to results.csv
